In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing

/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset, predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attributes):
    features = [privileged_attribute] + unprivileged_attributes

    aif_age_dataset = BinaryLabelDataset(
            df=dataset,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    aif_age_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    age_privileged_group = [{privileged_attribute: 1}]
    age_unprivileged_groups = [{attr: 1} for attr in unprivileged_attributes]

    fairness_metrics = ClassificationMetric(dataset=aif_age_dataset,
                            classified_dataset=aif_age_pred,
                            unprivileged_groups=age_unprivileged_groups,
                            privileged_groups=age_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")
    print(f"Average Odds Difference: {AOD}")

In [2]:
dataset_path = 'bankTrainedTidy.csv'
df_raw = pd.read_csv(dataset_path)
pd.set_option('display.max_columns', None)

In [4]:
X = df_raw.drop(columns="deposit")
y = df_raw["deposit"]

privileged_attributeAge = "age_25<=age<60"
unprivileged_attributesAge = ["age_age<25", "age_age>=60"]
age_features = [privileged_attributeAge] + unprivileged_attributesAge


In [5]:
privileged_group = [{privileged_attributeAge: 1}]
unprivileged_groups = [{attr: 1} for attr in unprivileged_attributesAge]

dataset = X.copy(deep=True) # we recreate a copy of the whole dataset processed and balanced
dataset['deposit'] = y # and join the target feature with the others

# Create a BinaryLabelDataset using AIF360 library
aif_dataset = BinaryLabelDataset(
        df=dataset,
        favorable_label=1,
        unfavorable_label=0,
        label_names=['deposit'],
        protected_attribute_names=age_features,
    )

# Apply the Reweighing technique
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_group)
dataset_transformed = RW.fit_transform(aif_dataset)

# Get sample weights from the transformed dataset
sample_weights = dataset_transformed.instance_weights

# Create a fair dataset using the reweighing technique
fair_dataset = dataset.copy(deep=True)
fair_dataset['weights'] = sample_weights

# Get features and target from the dataset
features = dataset.columns.tolist()
features.remove('deposit')
target = ['deposit']

# Set dataset features and target
X_fair = fair_dataset[features]
y_fair = fair_dataset[target]

X_fair_train, X_fair_test, y_fair_train, y_fair_test, sample_weights_train, sample_weights_test = train_test_split(X_fair, y_fair, sample_weights, test_size=0.2, random_state=42)

In [6]:
def train_models(sample_weight_tr):
    dt_clf = DecisionTreeClassifier(random_state=42)

    # The fit function will do the trick
    dt_clf.fit(X_fair_train, y_fair_train, sample_weight=sample_weight_tr)

    # After the training phase, the model will be tested by predicting the values on the test set
    dt_predictions = dt_clf.predict(X_fair_test)

    svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

    # Addestra il classificatore sui dati di training
    svm_classifier.fit(X_fair_train, y_fair_train, svc__sample_weight=sample_weight_tr)

    # Fai delle predizioni sui dati di test
    svm_pred = svm_classifier.predict(X_fair_test)

    rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

    rf_classifier.fit(X_fair_train, y_fair_train, sample_weight=sample_weight_tr)

    rf_predictions = rf_classifier.predict(X_fair_test)

    return dt_predictions,svm_pred,rf_predictions

In [7]:
dt_predictions, svm_pred, rf_predictions = train_models(sample_weights_train)

c:\Users\abasi\Desktop\Fairness_datasmell_SE4AI\.venv\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\abasi\Desktop\Fairness_datasmell_SE4AI\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [8]:
compute_performance_metrics(y_fair_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.7941818181818182
Decision Tree Recall: 0.7249544626593807
Decision Tree F1 Score: 0.7377201112140871


In [9]:
compute_performance_metrics(y_fair_test,svm_pred,"SVM")

SVM Accuracy: 0.8145454545454546
SVM Recall: 0.697632058287796
SVM F1 Score: 0.7502448579823702


In [10]:
compute_performance_metrics(y_fair_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.8501818181818181
Random Forest Recall: 0.7814207650273224
Random Forest F1 Score: 0.806390977443609


In [11]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_fair_test.copy(deep=True) # we create a copy of the test set
dataset['deposit'] = y_fair_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['deposit'] = dt_predictions # but this time the target feature is made by the predictions of our model

In [12]:
#Attributo privilegiato la colonna age_25<=age<60
#Attributo non privilegiato la colonna age_age<25, age_age>=60
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"deposit",1,0,privileged_attributeAge, unprivileged_attributesAge) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.377
Equal Opportunity Difference (EOD): 0.126
Average Odds Difference: 0.159


In [13]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_fair_test.copy(deep=True)
dataset['deposit'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['deposit'] = svm_pred

In [14]:
#Attributo privilegiato la colonna age_25<=age<60
#Attributo non privilegiato la colonna age_age<25, age_age>=60
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"deposit",1,0,privileged_attributeAge,unprivileged_attributesAge) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.23
Equal Opportunity Difference (EOD): -0.013
Average Odds Difference: -0.042


In [15]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_fair_test.copy(deep=True)
dataset['deposit'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['deposit'] = rf_predictions

In [16]:
#Attributo privilegiato la colonna age_25<=age<60
#Attributo non privilegiato la colonna age_age<25, age_age>=60
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"deposit",1,0,privileged_attributeAge,unprivileged_attributesAge) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.512
Equal Opportunity Difference (EOD): 0.217
Average Odds Difference: 0.253
